# CarND-Vehicle-Detection

## Goal

* Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier
* Optionally, you can also apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector.
* Note: for those first two steps don’t forget to normalize your features and randomize a selection for training and testing.
* Implement a sliding-window technique and use your trained classifier to search for vehicles in images.
* Run your pipeline on a video stream (start with the test_video.mp4 and later implement on full project_video.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
* Estimate a bounding box for vehicles detected.

## Setup

In [ ]:
import collections
import cv2
import glob
import math
import pickle
import scipy
import time

import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from tqdm import tqdm
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.ndimage.measurements import label
from moviepy.editor import VideoFileClip
from IPython.display import HTML

data_folder_ids = [1,2]
data_folder="../DataSets/data1/"
model_file = "./model.p"
scaler_file="./scaler.p"

RESIZE_H = 64
RESIZE_W = 64
RESIZE_TUPLE = (RESIZE_W, RESIZE_H)
SPATIAL_SIZE = (32, 32)
BINS_RANGE = (0, 255)
HIST_BINS = 32
ORIENT = 8
PIXELS_PER_CELL = 8
CELL_PER_BLOCK = 2
DEFAULT_COLOR_SPACE = 'YCrCb'
TIME_WINDOW = 20

%matplotlib inline

## Import Data

In [ ]:
vehicle_files = glob.glob(data_folder + 'vehicles/**/*.png', recursive=True)
non_vehicle_files = glob.glob(data_folder + 'non-vehicles/**/*.png', recursive=True)
print('Total Vehicle files : {}'.format(len(vehicle_files)))
print('Total non Vehicle files : {}'.format(len(non_vehicle_files)))

## Feature Training

In [ ]:
def extract_hog_features(image, orient=ORIENT, pix_per_cell=PIXELS_PER_CELL, cell_per_block=CELL_PER_BLOCK, vis=False,
feature_vec=True):
    if vis:
        features, hog_image = hog(image, orientations=orient,
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block),
                                  transform_sqrt=True,
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    else:
        features = hog(image, orientations=orient,
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block),
                       transform_sqrt=True,
                       visualise=vis, feature_vector=feature_vec)
        return features

def bin_spatial(image, size=SPATIAL_SIZE):
    features = cv2.resize(image, size).ravel()
    return features

def color_hist(image, nbins=HIST_BINS, bins_range=BINS_RANGE):
    channel1_hist = np.histogram(image[:, :, 0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(image[:, :, 1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(image[:, :, 2], bins=nbins, range=bins_range)
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    return hist_features

def extract_features_from_image(image, color_space=DEFAULT_COLOR_SPACE, spatial_size=SPATIAL_SIZE, hist_bins=HIST_BINS,
                                orient=ORIENT, pix_per_cell=PIXELS_PER_CELL,
                                cell_per_block=CELL_PER_BLOCK, hog_channel='ALL', spatial_feat=True,
                                hist_feat=True, hog_feat=True):
    image_features = []

    feature_image = convert_color(np.copy(image), 'HSV')
    feature_image[:, :, 2] += np.random.randint(0, 50)
    feature_image = cv2.cvtColor(feature_image, cv2.COLOR_HSV2BGR)
    feature_image = convert_color(np.copy(image), color_space)

    if spatial_feat:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        image_features.append(spatial_features)

    if hist_feat:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        image_features.append(hist_features)

    if hog_feat:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(extract_hog_features(feature_image[:, :, channel],
                                                     orient, pix_per_cell, cell_per_block,
                                                     vis=False, feature_vec=True))
            hog_features = np.ravel(hog_features)
        else:
            hog_features = extract_hog_features(feature_image[:, :, hog_channel], orient,
                                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        image_features.append(hog_features)

    return np.concatenate(image_features)

def extract_features_from_file_list(file_list):
    features = []
    for image_file in tqdm(file_list):
        resize_h, resize_w = 64, 64
        image = cv2.resize(cv2.imread(image_file), (resize_w, resize_h))
        file_features = extract_features_from_image(image)
        features.append(file_features)
    return features

def convert_color(image, dest_color_space=DEFAULT_COLOR_SPACE):
    if dest_color_space == 'YCrCb':
        image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    elif dest_color_space == 'YUV':
        image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    elif dest_color_space == 'LUV':
        image = cv2.cvtColor(image, cv2.COLOR_BGR2LUV)
    elif dest_color_space == 'HLS':
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    elif dest_color_space == 'HSV':
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    elif dest_color_space == 'grayscale':
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return image

class HOGClassifier:
    def __init__(self):
        self.svc = None
        self.X_test = None
        self.y_test = None
        self.X_scaler = None

    def train(self, vehicle_files, non_vehicle_files, test_size=0.2):
        vehicle_features = extract_features_from_file_list(vehicle_files)
        non_vehicle_features = extract_features_from_file_list(non_vehicle_files)
        X = np.vstack((vehicle_features, non_vehicle_features)).astype(np.float64)
        X_scaler = StandardScaler().fit(X)
        scaled_X = X_scaler.transform(X)
        y = np.hstack((np.ones(len(vehicle_features)), np.zeros(len(non_vehicle_features))))
        rand_state = np.random.randint(0, 100)
        X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=test_size, random_state=rand_state)
        svc = LinearSVC()
        svc.fit(X_train, y_train)
        print("Training complete")
        self.X_test = X_test
        self.y_test = y_test
        self.X_scaler = X_scaler
        self.svc = svc
        return svc

    def test(self, n_predict=15):
        print('Accuracy:     ', round(self.svc.score(self.X_test, self.y_test), 4))
        print('SVC Predicts: ', self.svc.predict(self.X_test[0:n_predict]))
        print('Labels:       ', self.y_test[0:n_predict])

    def dump(self, model_path, scaler_path):
        pickle.dump(self.svc, open(model_path, "wb"))
        pickle.dump(self.X_scaler, open(scaler_path, "wb"))
        print("End dump")

    def get(self, model_path, scaler_path):
        svc_trained = pickle.load(open(model_path, "rb"))
        xscaler = pickle.load(open(scaler_path, "rb"))
        return svc_trained, xscaler

classifier = HOGClassifier()
classifier.train(vehicle_files, non_vehicle_files)
classifier.test(n_predict=15)

In [ ]:
classifier.dump(model_file, scaler_file)

## Vehicle Detection

In [ ]:
svc_trained, xscaler = classifier.get(model_file, scaler_file)
hot_windows_history = collections.deque(maxlen=TIME_WINDOW)

def normalize_image(image):
    image = np.float32(image)
    image = image / image.max() * 255
    return np.uint8(image)

def gaussian_blur(image, kernel_size=5):
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

def draw_labeled_bounding_boxes(image, labeled_frame, num_objects):
    for car_number in range(1, num_objects + 1):
        rows, cols = np.where(labeled_frame == car_number)
        x_min, y_min = np.min(cols), np.min(rows)
        x_max, y_max = np.max(cols), np.max(rows)
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color=(255, 0, 0), thickness=6)
    return image

def draw_boxes(image, bbox_list, color=(0, 0, 255), thick=6):
    img_copy = np.copy(image)
    for bbox in bbox_list:
        tl_corner = tuple(bbox[0])
        br_corner = tuple(bbox[1])
        cv2.rectangle(img_copy, tl_corner, br_corner, color, thick)
    return img_copy

def compute_heatmap_from_detections(frame, hot_windows, threshold=5, verbose=False):
    h, w, c = frame.shape

    heatmap = np.zeros(shape=(h, w), dtype=np.uint8)

    for bbox in hot_windows:
        # for each bounding box, add heat to the corresponding rectangle in the image
        x_min, y_min = bbox[0]
        x_max, y_max = bbox[1]
        heatmap[y_min:y_max, x_min:x_max] += 1  # add heat

    # apply threshold + morphological closure to remove noise
    _, heatmap_thresh = cv2.threshold(heatmap, threshold, 255, type=cv2.THRESH_BINARY)
    heatmap_thresh = cv2.morphologyEx(heatmap_thresh, op=cv2.MORPH_CLOSE,
                                      kernel=cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (13, 13)), iterations=1)
    if verbose:
        f, ax = plt.subplots(1, 3)
        ax[0].imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        ax[1].imshow(heatmap, cmap='hot')
        ax[2].imshow(heatmap_thresh, cmap='hot')
        plt.show()

    return heatmap, heatmap_thresh

def search_vehicles(image, y_start, y_stop, scale, svc, feature_scaler, color_space=DEFAULT_COLOR_SPACE,
                  spatial_size=SPATIAL_SIZE, hist_bins=HIST_BINS, orient=ORIENT, pix_per_cell=PIXELS_PER_CELL,
                  cell_per_block=CELL_PER_BLOCK):
    hot_windows = []
    resize_h = RESIZE_H
    resize_w = RESIZE_W
    draw_img = np.copy(image)
    image_crop = draw_img[y_start:y_stop, :, :]
    image_crop = convert_color(image_crop, dest_color_space=color_space)

    if scale != 1:
        imshape = image_crop.shape
        image_crop = cv2.resize(image_crop, (np.int(imshape[1] / scale), np.int(imshape[0] / scale)))

    ch1 = image_crop[:, :, 0]
    ch2 = image_crop[:, :, 1]
    ch3 = image_crop[:, :, 2]
    n_x_blocks = (ch1.shape[1] // pix_per_cell) - 1
    n_y_blocks = (ch1.shape[0] // pix_per_cell) - 1

    window = 64
    n_blocks_per_window = (window // pix_per_cell) - 1
    cells_per_step = 4
    n_x_steps = (n_x_blocks - n_blocks_per_window) // cells_per_step
    n_y_steps = (n_y_blocks - n_blocks_per_window) // cells_per_step
    hog1 = extract_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = extract_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = extract_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)

    for xb in range(n_x_steps):
        for yb in range(n_y_steps):
            y_pos = yb * cells_per_step
            x_pos = xb * cells_per_step
            hog_feat1 = hog1[y_pos:y_pos + n_blocks_per_window, x_pos:x_pos + n_blocks_per_window].ravel()
            hog_feat2 = hog2[y_pos:y_pos + n_blocks_per_window, x_pos:x_pos + n_blocks_per_window].ravel()
            hog_feat3 = hog3[y_pos:y_pos + n_blocks_per_window, x_pos:x_pos + n_blocks_per_window].ravel()
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))
            x_left = x_pos * pix_per_cell
            y_top = y_pos * pix_per_cell
            subimg = cv2.resize(image_crop[y_top:y_top + window, x_left:x_left + window], (resize_w, resize_h))
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)
            test_features = feature_scaler.transform(
                np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))
            test_prediction = svc.decision_function(test_features)

            if test_prediction > 0.2 and test_prediction <= 1:
                xbox_left = np.int(x_left * scale)
                ytop_draw = np.int(y_top * scale)
                win_draw = np.int(window * scale)
                tl_corner_draw = (xbox_left, ytop_draw + y_start)
                br_corner_draw = (xbox_left + win_draw, ytop_draw + win_draw + y_start)
                cv2.rectangle(draw_img, tl_corner_draw, br_corner_draw, (0, 0, 255), 6)
                hot_windows.append((tl_corner_draw, br_corner_draw))

    return hot_windows

def process_image(frame, svc=None, scaler=None, keep_state=True, verbose=False):
    hot_windows = []

    if verbose is False:
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    frame_blurred = gaussian_blur(frame, 3)

    for size_subsample in np.arange(1, 4, .3):
        hot_windows += search_vehicles(frame_blurred, 400, 600, size_subsample, svc, scaler)

    if keep_state:
        if hot_windows:
            hot_windows_history.append(hot_windows)
            hot_windows = np.concatenate(hot_windows_history)

    thresh = TIME_WINDOW if keep_state else 1
    heatmap, heatmap_thresh = compute_heatmap_from_detections(frame, hot_windows, threshold=thresh, verbose=False)

    img_thresh = np.zeros(heatmap_thresh.shape, dtype=np.uint8)
    img_blur = gaussian_blur(heatmap_thresh, 21)
    img_thresh[img_blur > 50] = 255

    img_hot_windows = draw_boxes(frame, hot_windows, color=(0, 0, 255), thick=2)
    img_heatmap = cv2.applyColorMap(normalize_image(heatmap), colormap=cv2.COLORMAP_HOT)
    img_labeling = normalize_image(img_thresh)
    _, contours, _ = cv2.findContours(img_thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    boxes = []
    for i, contour in enumerate(contours):
        rectangle = cv2.boundingRect(contour)
        x, y, w, h = rectangle
        boxes.append([(x, y), (x + w, y + h)])

    img_detection = draw_boxes(frame, boxes, thick=2, color=(255, 0, 0))
    return cv2.cvtColor(img_detection, cv2.COLOR_BGR2RGB)

def process_video(input_file="project_video.mp4", output_file='output_video.mp4', svc=None, scaler=None,
                   keep_state=True, verbose=False):
    clip1 = VideoFileClip(input_file)

    def _process_image(frame, svc=svc, scaler=scaler, keep_state=keep_state, verbose=verbose):
        return process_image(frame, svc=svc, scaler=scaler, keep_state=keep_state, verbose=verbose)

    out_clip = clip1.fl_image(_process_image)
    return out_clip.write_videofile(output_file, audio=False, verbose=True, progress_bar=False)

## Video Process

In [ ]:
%time process_video(svc=svc_trained, scaler=xscaler, keep_state=True, verbose=False)